In [150]:
import pandas as pd
import numpy as np

In [151]:
lines = pd.read_csv("the_office_lines.csv")
lines.tail(6)

,id,season,episode,scene,line_text,speaker,deleted
59903,59904,9,23,111,"Everyday when I came into work, all I wanted t...",Darryl,False
59904,59905,9,23,112,It all seems so very arbitrary. I applied for ...,Creed,False
59905,59906,9,23,113,I just feel lucky that I got a chance to share...,Meredith,False
59906,59907,9,23,114,I���m happy that this was all filmed so I can ...,Phyllis,False
59907,59908,9,23,115,I sold paper at this company for 12 years. My ...,Jim,False
59908,59909,9,23,116,I thought it was weird when you picked us to m...,Pam,False


In [152]:
lines.dtypes

id            int64
season        int64
episode       int64
scene         int64
line_text    object
speaker      object
deleted        bool
dtype: object

In [153]:
lines.isnull().sum()

id           0
season       0
episode      0
scene        0
line_text    0
speaker      0
deleted      0
dtype: int64

In [154]:
#elimino escenas borradas
lines = lines[(lines["deleted"] == False)]
lines.head()

,id,season,episode,scene,line_text,speaker,deleted
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False


In [155]:
lines.shape

(57973, 7)

In [156]:
lines['speaker'] = lines.speaker.str.lower()

In [157]:
lines.loc[[5]]

,id,season,episode,scene,line_text,speaker,deleted
5,6,1,1,2,"[on the phone] Yes, I'd like to speak to your ...",michael,False


In [158]:
lines["action"] = lines["line_text"].str.extract("(\s*\[.*?\]\s*)", expand=True)
lines["line_text"] = lines["line_text"].str.replace(r"(\s*\[.*?\]\s*)", " ", regex=True)

In [159]:
lines.loc[[5]]

,id,season,episode,scene,line_text,speaker,deleted,action
5,6,1,1,2,"Yes, I'd like to speak to your office manager...",michael,False,[on the phone]


In [160]:
lines["speaker"] = lines["speaker"].str.replace(r"(\s*\[.*?\]\s*)", " ", regex=True)

In [161]:
lines["line_text"] = lines["line_text"].str.replace("���", "'")

In [162]:
lines["line_text"] = lines["line_text"].replace('%','%%', regex=True)

In [163]:
lines.tail(6)

,id,season,episode,scene,line_text,speaker,deleted,action
59903,59904,9,23,111,"Everyday when I came into work, all I wanted t...",darryl,False,NaN
59904,59905,9,23,112,It all seems so very arbitrary. I applied for ...,creed,False,[chuckles]
59905,59906,9,23,113,I just feel lucky that I got a chance to share...,meredith,False,NaN
59906,59907,9,23,114,I'm happy that this was all filmed so I can re...,phyllis,False,NaN
59907,59908,9,23,115,I sold paper at this company for 12 years. My ...,jim,False,NaN
59908,59909,9,23,116,I thought it was weird when you picked us to m...,pam,False,NaN


In [164]:
lines["speaker"] = lines["speaker"].str.replace('[a-zA-Z]*mic[a-zA-Z]*', 'michael', regex=True)

In [165]:
df_obj = lines.select_dtypes(['object'])

lines[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [166]:
lines["speaker"].unique()

array(['michael', 'jim', 'pam', 'dwight', 'jan', 'todd packer', 'phyllis',
       'stanley', 'oscar', 'angela', 'kevin', 'ryan', 'man', 'roy',
       'mr. brown', 'toby', 'kelly', 'meredith', 'travel agent',
       'man on phone', 'everybody', 'lonny', 'darryl', 'teammates',
       'michael and dwight', 'warehouse worker', 'madge', 'worker',
       'katy', 'guy at bar', 'other guy at bar', 'pam and jim',
       'employee', "chili's employee", "kevin's computer",
       'warehouse guy', 'warehouse guys', 'video', 'man in video',
       'actor', 'redheaded actress', "mr. o'malley", 'albiny',
       "pam's mom", 'carol', 'bill', 'everyone', 'crowd', 'song',
       'dwight and michael', 'sherri', 'creed', 'devon', 'children',
       'kid', 'hank the security guard', 'ira', "ryan's voicemail",
       'master', 'christian', 'hostess', 'michael and christian',
       'sadiq (it guy)', 'mark', 'improv teacher', 'mary-beth',
       'girl acting pregnant', 'actress', 'michael and jim',
       'k

In [167]:
lines["speaker"] = lines["speaker"].str.capitalize()
lines.head()

,id,season,episode,scene,line_text,speaker,deleted,action
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,NaN
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,NaN
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,NaN
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,NaN
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,NaN


In [168]:
lines = lines.rename(columns = {"id": "line_id", "speaker": "first_name"})
lines.head()

,line_id,season,episode,scene,line_text,first_name,deleted,action
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,NaN
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,NaN
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,NaN
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,NaN
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,NaN


In [171]:
lines1 = lines.groupby(['season', 'episode']).size().reset_index(name='freq')
lines1.head()

,season,episode,freq
0,1,1,229
1,1,2,217
2,1,3,244
3,1,4,265
4,1,5,235


In [172]:
lines1.index = lines1.index.set_names(["episode_id"])
lines1 = lines1.reset_index()
lines1.head()

,episode_id,season,episode,freq
0,0,1,1,229
1,1,1,2,217
2,2,1,3,244
3,3,1,4,265
4,4,1,5,235


In [173]:
lines1.tail()

,episode_id,season,episode,freq
181,181,9,19,273
182,182,9,20,234
183,183,9,21,382
184,184,9,22,501
185,185,9,23,522


In [174]:
left_merged = pd.merge(lines,lines1, how="left", on=["season", "episode"])

In [175]:
left_merged.head()

,line_id,season,episode,scene,line_text,first_name,deleted,action,episode_id,freq
0,1,1,1,1,All right Jim. Your quarterlies look very good...,Michael,False,NaN,0,229
1,2,1,1,1,"Oh, I told you. I couldn't close it. So...",Jim,False,NaN,0,229
2,3,1,1,1,So you've come to the master for guidance? Is ...,Michael,False,NaN,0,229
3,4,1,1,1,"Actually, you called me in here, but yeah.",Jim,False,NaN,0,229
4,5,1,1,1,"All right. Well, let me show you how it's done.",Michael,False,NaN,0,229


In [176]:
left_merged = left_merged.drop("freq", axis=1)

In [178]:
left_merged.tail()

,line_id,season,episode,scene,line_text,first_name,deleted,action,episode_id
57968,59905,9,23,112,It all seems so very arbitrary. I applied for ...,Creed,False,[chuckles],185
57969,59906,9,23,113,I just feel lucky that I got a chance to share...,Meredith,False,NaN,185
57970,59907,9,23,114,I'm happy that this was all filmed so I can re...,Phyllis,False,NaN,185
57971,59908,9,23,115,I sold paper at this company for 12 years. My ...,Jim,False,NaN,185
57972,59909,9,23,116,I thought it was weird when you picked us to m...,Pam,False,NaN,185


In [179]:
left_merged.to_csv('lines.csv', index=False)